In [13]:
import requests
import numpy as np
from LSBSteg import encode
from riddle_solvers import *
import cv2
from PIL import Image
# api_base_url = "http://3.70.97.142:5000"
api_base_url = "http://localhost:3005"
# team_id = Lu2xdzj (take care to use the same team id and start game 🧑🏼‍🚒)
# team_id="Lu2xdzj"
team_id = "xxx"
total_budget=0
def init_fox(team_id):
    '''
    In this fucntion you need to hit to the endpoint to start the game as a fox with your team id.
    If a sucessful response is returned, you will recive back the message that you can break into chunkcs
      and the carrier image that you will encode the chunk in it.
    '''
    payload_sent = {
        'teamId': team_id
    }
    response = requests.post(api_base_url+"/fox/start", json=payload_sent)
    print(response)
    if response.status_code == 200 or response.status_code == 201:
        print("Game started successfully")
        data = response.json()
        msg = data['msg']
        carrier_image = data['carrier_image']
        return msg, np.array(carrier_image),np.array(carrier_image,dtype=np.uint8)
    else:
        print("error: ", response.status_code)

def generate_message_array(message, image_carrier,total_budget):  
    '''
    In this function you will need to create your own startegy. That includes:
        1. How you are going to split the real message into chunkcs
        2. Include any fake chunks
        3. Decide what 3 chuncks you will send in each turn in the 3 channels & what is their entities (F,R,E)
        4. Encode each chunck in the image carrier  
    '''
    def split_string_into_two_chars(input_string):
        pairs = [input_string[i:i+2] for i in range(0, len(input_string), 2)]
        if len(input_string) % 2 != 0:
            pairs.append(pairs.pop()[0])
        return pairs
    # new_message = split_string_into_two_chars(message)
    new_message =[message]
    index=0
    channel=0
    def sent_message(fake_msg,real_msg,total_budget,channel):
        message_entities = ['E' for _ in range(3)]
        messages = [image_carrier.tolist() for _ in range(3)]
        for i in range(len(fake_msg)):
            image=encode(image_carrier.copy(),fake_msg[i]).tolist()
            messages[channel]=image
            message_entities[channel]='F'
            channel=(channel+1)%3
            total_budget-=1
        
        for i in range(len(real_msg)):
            image=encode(image_carrier.copy(),real_msg[i]).tolist()
            messages[channel]=image
            message_entities[channel]='R'
            channel=(channel+1)%3
        send_message(team_id, messages, message_entities)
        return channel,total_budget

    if(total_budget>=2):
        print("total_budget > 2")
        channel,total_budget = sent_message(["fake","fffff"],[new_message[index]],total_budget,channel)
        
    elif (total_budget>=1):
        print("total_budget > 1")
        channel,total_budget = sent_message(["fake"],[new_message[index]],total_budget,channel)
    else:
        print("total_budget == 0")
        channel,total_budget = sent_message([],[new_message[index]],total_budget,channel)
    # while(total_budget>0 ):
    #     if(total_budget>=2):
    #         sent_message(["fake","fffff"],[new_message[index]])
    #     else:
    #         sent_message(["fake"],[new_message[index]])
    #     index+=1
    # while(index< len(new_message)):
    #     sent_message([],[new_message[index]])
    #     index+=1


def get_riddle(team_id, riddle_id):
    '''
    In this function you will hit the api end point that requests the type of riddle you want to solve.
    use the riddle id to request the specific riddle.
    Note that: 
        1. Once you requested a riddle you cannot request it again per game. 
        2. Each riddle has a timeout if you didnot reply with your answer it will be considered as a wrong answer.
        3. You cannot request several riddles at a time, so requesting a new riddle without answering the old one
          will allow you to answer only the new riddle and you will have no access again to the old riddle. 
    '''
    payload_sent = {
        'teamId': team_id,
        "riddleId": riddle_id
    }
    response = requests.post(api_base_url+"/fox/get-riddle", json=payload_sent)
    if response.status_code == 200 or response.status_code == 201:
        data = response.json()
        print("Riddle requested successfully")
        test_case = data['test_case']
        return test_case
    else:
        print("error: ", response.status_code)
        return ''

def solve_riddle(team_id, solution,total_budget):
    '''
    In this function you will solve the riddle that you have requested. 
    You will hit the API end point that submits your answer.
    Use te riddle_solvers.py to implement the logic of each riddle.
    '''
    payload_sent = {
        'teamId': team_id,
        "solution": solution
    }
    response = requests.post(api_base_url+"/fox/solve-riddle", json=payload_sent)
    if response.status_code == 200 or response.status_code == 201:
        data = response.json()
        budget_increase = data['budget_increase']
        total_budget = data['total_budget']
        status = data['status']
        if(status == "success"):
            print("Riddle solved successfully")
            print("Budget increased by: ", budget_increase)
            print("Total budget: ", total_budget)
        else:
            print("Riddle not solved")
    else:
        print("error: ", response.status_code)
    return total_budget

def send_message(team_id, messages, message_entities=['F', 'E', 'R']):
    '''
    Use this function to call the api end point to send one chunk of the message. 
    You will need to send the message (images) in each of the 3 channels along with their entites.
    Refer to the API documentation to know more about what needs to be send in this api call. 
    '''
    payload_sent = {
        'teamId': team_id,
        "messages": messages,
        "message_entities":message_entities
    }
    print("das")
    response = requests.post(api_base_url+"/fox/send-message", json=payload_sent)
    if response.status_code == 200 or response.status_code == 201:
       print("Message sent successfully")
    else:
        print("error: ", response.status_code)
   
def end_fox(team_id):
    '''
    Use this function to call the api end point of ending the fox game.
    Note that:
    1. Not calling this fucntion will cost you in the scoring function
    2. Calling it without sending all the real messages will also affect your scoring fucntion
      (Like failing to submit the entire message within the timelimit of the game).
    '''
    payload_sent = {
        'teamId': team_id,
    }
    response = requests.post(api_base_url+"/fox/end-game", json=payload_sent)
    if response.status_code == 200 or response.status_code == 201:
        print("Game ended successfully")
    else:
        print("error: ", response.status_code)

# def submit_fox_attempt(team_id,total_budget):
#     '''
#      Call this function to start playing as a fox. 
#      You should submit with your own team id that was sent to you in the email.
#      Remeber you have up to 15 Submissions as a Fox In phase1.
#      In this function you should:
#         1. Initialize the game as fox 
#         2. Solve riddles 
#         3. Make your own Strategy of sending the messages in the 3 channels
#         4. Make your own Strategy of splitting the message into chunks
#         5. Send the messages 
#         6. End the Game
#     Note that:
#         1. You HAVE to start and end the game on your own. The time between the starting and ending the game is taken into the scoring function
#         2. You can send in the 3 channels any combination of F(Fake),R(Real),E(Empty) under the conditions that
#             2.a. At most one real message is sent
#             2.b. You cannot send 3 E(Empty) messages, there should be atleast R(Real)/F(Fake)
#         3. Refer To the documentation to know more about the API handling 
#     '''
#     msg, carrier_image=init_fox(team_id)
#     ## solve riddles
#     try:
#         for riddle_id,riddle_func in riddle_solvers.items():
#             test_case = get_riddle(team_id, riddle_id)
#             if test_case == '':
#                 continue
#             solution = riddle_func(test_case)
#             solve_riddle(team_id, solution,total_budget)
#     except Exception as e:
#         print(e)
#     total_budget=min(total_budget,12)
#     ## generate message array
#     generate_message_array(msg, carrier_image)
#     end_fox(team_id)


# submit_fox_attempt(team_id,total_budget)


In [14]:
msg, carrier_image,carrier_image2=init_fox(team_id)

<Response [201]>
Game started successfully


In [15]:
riddle_id="problem_solving_hard"
test_case = get_riddle(team_id, riddle_id)
solution = solve_problem_solving_hard(test_case)
total_budget = solve_riddle(team_id, solution,total_budget)


Riddle requested successfully
Riddle solved successfully
Budget increased by:  3
Total budget:  3


In [16]:
generate_message_array(msg, carrier_image,total_budget)

total_budget > 2


In [20]:
riddle_id="sec_hard"
test_case = get_riddle(team_id, riddle_id)
solution = solve_sec_hard(test_case)
total_budget = solve_riddle(team_id, solution,total_budget)

try:
    riddle_id="sec_medium"
    test_case = get_riddle(team_id, riddle_id)
    solution = solve_sec_medium(test_case)
    total_budget = solve_riddle(team_id, solution,total_budget)
except Exception as e:
    print('error in sec_medium')
    print(e)

error:  400
error in sec_medium
local variable 'img' referenced before assignment


In [18]:
riddle_id="problem_solving_easy"
test_case = get_riddle(team_id, riddle_id)
solution = solve_problem_solving_easy(test_case)
total_budget = solve_riddle(team_id, solution,total_budget)



riddle_id="problem_solving_medium"
test_case = get_riddle(team_id, riddle_id)
solution = solve_problem_solving_medium(test_case)
total_budget = solve_riddle(team_id, solution,total_budget)

error:  400


ValueError: not enough values to unpack (expected 2, got 0)

In [19]:
riddle_id="cv_medium"
test_case = get_riddle(team_id, riddle_id)
solution = solve_cv_medium(test_case)
total_budget = solve_riddle(team_id, solution,total_budget)



error:  400
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\ahmed\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ahmed\AppData\Local\Temp\ipykernel_7492\3767247619.py", line 3, in <module>
    solution = solve_cv_medium(test_case)
  File "c:\TempDesktop\hacktrick\Cipher-Sphere\Solvers\riddle_solvers.py", line 58, in solve_cv_medium
ValueError: not enough values to unpack (expected 2, got 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ahmed\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\ahmed\AppData\Roaming\Python\Python39\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\ahmed\

In [21]:

riddle_id="cv_easy"
test_case = get_riddle(team_id, riddle_id)
solution = solve_cv_easy(test_case)
solve_riddle(team_id, solution,total_budget)

error:  400
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\ahmed\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ahmed\AppData\Local\Temp\ipykernel_7492\3747088834.py", line 3, in <module>
    solution = solve_cv_easy(test_case)
  File "c:\TempDesktop\hacktrick\Cipher-Sphere\Solvers\riddle_solvers.py", line 10, in solve_cv_easy
ValueError: not enough values to unpack (expected 2, got 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ahmed\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\ahmed\AppData\Roaming\Python\Python39\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\ahmed\AppD

In [ ]:
try:
    riddle_id="ml_easy"
    test_case = get_riddle(team_id, riddle_id)
    solution = solve_ml_easy(test_case)
    solve_riddle(team_id, solution,total_budget)
except Exception as e:
    print('error in ml_easy')
    print(e)

In [22]:
end_fox("Lu2xdzj")

Game ended successfully
